
# <center><font color= #F08080 > <b>CU45_Planificación y promoción del destino en base a los patrones en origen de los turistas</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

## Modelo CLUSTER



### Paquetes

In [1]:


library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(purrr)
library(tibble)




Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




### Datos

In [2]:
receptor <- read_csv("CU_45_05_03_receptor.csv")
interno_prov <- read_csv("CU_45_05_04_interno_prov.csv")
interno_muni <- read_csv("CU_45_05_05_interno_mun.csv")
valoraciones <- read_csv("CU_45_05_02_valoracion_sim.csv")



Rows: 50294 Columns: 7
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (5): mes, pais_orig_cod, pais_orig, mun_dest, CMUN
dbl (2): mun_dest_cod, turistas

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 521280 Columns: 8
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (6): total_nacional, total_ccaa, provincia, municipio_destino, cmun, mes
dbl (2): turistas, secreto

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 79246 Columns: 10
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (7): mes, mun_orig_cod, mun_orig, dest, prov_orig_cod, prov_orig, prov_dest
dbl (3): dest_cod, turistas, prov_dest_cod

i Use `spec()` t

### Clustering

In [3]:
turistas_origen_mes <- 
  receptor |> 
  filter(pais_orig == "Total") |> 
  select(mes, CMUN, mun_dest, turistas) |> 
  mutate(receptor = turistas) |> 
  select(-turistas) |> 
  inner_join(
    interno_prov |> 
      filter(!is.na(total_ccaa),
             is.na(provincia)) |> 
      select(mes, cmun, total_ccaa, turistas) |> 
      pivot_wider(names_from = total_ccaa, 
                  values_from = turistas,
                  values_fn = ~sum(.x, na.rm = TRUE)),
    by = c("CMUN" = "cmun", "mes" = "mes"))

val <- valoraciones |> 
  group_by(CMUN, grupo) |> 
  summarise(puntos = median(puntos, na.rm = TRUE),.groups = "drop") |> 
  pivot_wider(names_from = grupo,
              values_from = puntos) |> 
  mutate(across(2:4, ~replace_na(.x, median(.x, na.rm = TRUE))))

kdata <- turistas_origen_mes |> 
  inner_join(val, by = "CMUN") |> 
  # select(-municipio_destino) |> 
  mutate(anyo = str_sub(mes, 1, 4)) |> 
  group_by(anyo, mun_dest) |> 
  summarise(across(3:(ncol(turistas_origen_mes) - 1), sum),
            across((ncol(turistas_origen_mes) - 1):(ncol(turistas_origen_mes) + 2), median))

by_anyo <- split(kdata, kdata$anyo)

by_anyo[[1]] |> 
  column_to_rownames("mun_dest") |> 
  select(-anyo) |> 
  kmeans(10)


## PARÁMETRO DEL PASO 2
NCLUS <- 4

cluster_anyos <- map(by_anyo, function(x)  {
  m <- x |> 
    column_to_rownames("mun_dest") |> 
    select(-anyo) |> 
    kmeans(NCLUS)
  res <- x |> 
    mutate(cluster = m$cluster)
}) |> bind_rows() |> 
  ungroup()

write_csv(cluster_anyos, "cluster_anyos.csv")



`summarise()` has grouped output by 'anyo'. You can override using the
`.groups` argument.


K-means clustering with 10 clusters of sizes 11, 18, 11, 15, 3, 14, 36, 10, 25, 1

Cluster means:
       receptor        Andaluc<U+00ED>a        Arag<U+00F3>n Asturias, Principado de
1    12363.3636       2526.46145  1083.2727273             1379.272727
2      770.1111        344.38889    42.4444444               42.500000
3     5386.8182       3373.94927   603.7272727              921.000000
4     1802.0667       1085.13333   236.6000000              257.333333
5    47009.3333         28.35633  2794.3226667             3549.771667
6    23805.7143         15.82829  2393.6428571             2870.594786
7      398.8611        102.11111     0.9166667                1.944444
8     2002.8000       1130.40000   207.0000000              133.700000
9     1167.8800        765.20000    52.2400000               91.720000
10 2602822.0000        854.18400   163.6530000              177.368000
   Balears, Illes     Canarias   Cantabria Castilla - La Mancha
1     690.1818182  877.8181818  745.545455 